In [1]:
import wave
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
from tqdm import tqdm
from scipy.io import wavfile
import librosa
import matplotlib.image as mpimg 

In [3]:
from PIL import Image

In [4]:
from python_speech_features import mfcc,logfbank

In [5]:
from keras.layers import Conv2D, MaxPool2D, Flatten, LSTM
from keras.layers import Dropout, Dense, TimeDistributed
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [3]:
os.chdir('C:/Users/Admin/Desktop/Respiratory_Sound_Database/Respiratory_Sound_Database')

In [7]:
df=pd.read_csv("label.csv")

In [8]:
df.head()

,fname,label,length
0,101_1b1_Al_sc_Meditron0,Healthy,0.542993
1,101_1b1_Al_sc_Meditron1,Healthy,1.871020
2,101_1b1_Al_sc_Meditron2,Healthy,1.442993
3,101_1b1_Al_sc_Meditron3,Healthy,1.900000
4,101_1b1_Al_sc_Meditron4,Healthy,1.728005


In [9]:
df.set_index('fname', inplace=True)

In [10]:
x=[]
y=[]
_min,_max = float('inf'),-float('inf')
for _ in tqdm(range(20000)):
    file = np.random.choice(df[df.label=='Healthy'].index)
    rate, wav = wavfile.read('clean/'+file+'.wav')
    if len(wav)>20000:
        label = df.at[file,'label']
        rand_index = np.random.randint(0,wav.shape[0]-20000)
        sample = wav[rand_index:rand_index+20000]
        x_sample = mfcc(sample,rate,numcep=26,nfilt=26,nfft=512)
        _min = min(np.amin(x_sample), _min)
        _max = max(np.amax(x_sample), _max)
        x.append(x_sample)
        y.append(0)

100%|███████████████████████████████████████████████████████████████████████████| 20000/20000 [02:17<00:00, 145.19it/s]


In [11]:
_min,_max = float('inf'),-float('inf')
for _ in tqdm(range(20000)):
    file = np.random.choice(df[df.label=='Crackles'].index)
    rate, wav = wavfile.read('clean/'+file+'.wav')
    if len(wav)>20000:
        label = df.at[file,'label']
        rand_index = np.random.randint(0,wav.shape[0]-20000)
        sample = wav[rand_index:rand_index+20000]
        x_sample = mfcc(sample,rate,numcep=26,nfilt=26,nfft=512)
        _min = min(np.amin(x_sample), _min)
        _max = max(np.amax(x_sample), _max)
        x.append(x_sample)
        y.append(1)

100%|███████████████████████████████████████████████████████████████████████████| 20000/20000 [02:47<00:00, 119.25it/s]


In [12]:
_min,_max = float('inf'),-float('inf')
for _ in tqdm(range(20000)):
    file = np.random.choice(df[df.label=='Wheezes'].index)
    rate, wav = wavfile.read('clean/'+file+'.wav')
    if len(wav)>20000:
        label = df.at[file,'label']
        rand_index = np.random.randint(0,wav.shape[0]-20000)
        sample = wav[rand_index:rand_index+20000]
        x_sample = mfcc(sample,rate,numcep=26,nfilt=26,nfft=512)
        _min = min(np.amin(x_sample), _min)
        _max = max(np.amax(x_sample), _max)
        x.append(x_sample)
        y.append(2)

100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [03:34<00:00, 93.36it/s]


In [13]:
x, y = np.array(x), np.array(y)

In [15]:
x = (x - _min)/(_max - _min)

In [16]:
x.shape

(55370, 124, 26)

In [17]:
x = x.reshape(x.shape[0],x.shape[1],x.shape[2],1)

In [18]:
x.shape

(55370, 124, 26, 1)

In [19]:
y.shape

(55370,)

In [20]:
y= to_categorical(y, num_classes=3)

In [21]:
y.shape

(55370, 3)

In [22]:
input_shape = (124,26,1)

In [23]:
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', strides=(1,1),
                padding= "same", input_shape=input_shape))
model.add(Conv2D(32, (3,3), activation='relu', strides=(1,1),
                padding='same'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu', strides=(1,1),
                 padding='same'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(128, (3,3), activation='relu', strides=(1,1),
                padding='same'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(256, (3,3), activation='relu', strides=(1,1),
                padding='same'))
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['acc'])

W0921 00:48:45.745438 15084 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:68: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0921 00:48:46.584354 15084 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:504: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0921 00:48:46.765267 15084 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3828: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0921 00:48:46.944133 15084 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3652: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0921 00:48:46.995995 15084 deprecation_wrapper.py:119] From C:\Users\Admin\Ana

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 124, 26, 16)       160       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 26, 32)       4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 62, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 6, 64)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 31, 6, 128)        73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 3, 128)        0         
__________

W0921 00:48:47.323121 15084 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\optimizers.py:744: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0921 00:48:47.350493 15084 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3005: The name tf.log is deprecated. Please use tf.math.log instead.



In [ ]:
model.fit(x,y, epochs=20, batch_size=16 , validation_split=0.1)

In [30]:
model.predict(x[21000:21010])

array([[0.33250028, 0.48060024, 0.18689954],
       [0.3844606 , 0.28842393, 0.3271155 ],
       [0.17454538, 0.8198472 , 0.00560739],
       [0.35802892, 0.5604923 , 0.08147873],
       [0.39403057, 0.29894182, 0.30702764],
       [0.5096481 , 0.34692943, 0.14342247],
       [0.03362099, 0.7695902 , 0.19678877],
       [0.36162734, 0.3673059 , 0.27106676],
       [0.19498323, 0.727334  , 0.07768276],
       [0.23435824, 0.70468545, 0.06095637]], dtype=float32)

In [ ]:
model.save('model.h5')